# MINI PROJECT(190819)
## CIFAR-100 분류하기

In [ ]:
# need library
import numpy as np
import tensorflow as tf

# keras
from keras.models import Sequential
from keras.layers import Dense

# warning remove
if type(tf.contrib) != type(tf):
    tf.contrib._warning = None

In [ ]:
# 데이터 전처리
from keras.datasets import cifar100
(X_train, y_train0), (X_test, y_test0) = cifar100.load_data()
print(X_train.shape, X_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test.shape, X_test.dtype)
print(y_test0.shape, y_test0.dtype)

In [ ]:
# 데이터 확인
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.subplot(141)
plt.imshow(X_train[0], interpolation="bicubic")
plt.grid(False)
plt.subplot(142)
plt.imshow(X_train[4], interpolation="bicubic")
plt.grid(False)
plt.subplot(143)
plt.imshow(X_train[8], interpolation="bicubic")
plt.grid(False)
plt.subplot(144)
plt.imshow(X_train[12], interpolation="bicubic")
plt.grid(False)
plt.show()

In [ ]:
# 데이터 변환 및 정규화
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0
print(X_train.shape, X_train.dtype)
print(X_test.shape, X_test.dtype)

In [ ]:
# one-hot-encoding
print(min(y_train0),max(y_train0))

In [ ]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train0, 100)
Y_test = np_utils.to_categorical(y_test0, 100)
Y_train[:2]

In [ ]:
# modeling
import os
import sys

# keras
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [ ]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

In [ ]:
# 컨볼루션 신경망의 설정
model = Sequential()

In [ ]:
# 마스크 갯수, 마스크의 크기, 입력되는 값 및 색상, 활성화 함수
model.add(Conv2D(128, kernel_size=(5, 5), input_shape=(32, 32, 3), padding='same', activation='relu')) # input_shape=(행, 열, 색상(3) 또는 흑백(1))
model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='softplus'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='relu')) 
model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='softplus'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Dropout(0.25)) 
model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='relu')) 
model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='softplus'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Dropout(0.25)) 
model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='relu')) 
model.add(Conv2D(128, kernel_size=(5, 5), padding='same', activation='softplus'))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Dropout(0.25)) 
model.add(Flatten())
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='softmax'))
model.summary()

In [ ]:
# model compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 모델 최적화 설정
MODEL_DIR = 'model100//'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
modelpath="model100//{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=50)

In [ ]:
# training
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=1000, batch_size=10, verbose=1, 
                    callbacks=[early_stopping_callback,checkpointer])

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

In [ ]:
# 그래프 
import matplotlib.pyplot as plt
%matplotlib inline
y_vloss = history.history['val_loss'] # 검증셋으로 측정한 오차
y_loss = history.history['loss'] # 학습셋의 오차
x_len = np.arange(1,len(y_loss)+1)

plt.figure(figsize=(8,8))
plt.plot(x_len, y_vloss, marker='.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='Trainset_loss')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('cifar10 data (CNN)')
plt.show()